# Deploy a employee promotion prediction model to SAP AI Core

In this notebook we deploy the model to [SAP AI Core](https://help.sap.com/docs/sap-ai-core) as a custom ML engine IBM Watson OpenScale. We will go through the steps to authenticate to SAP AI Core, add secrets for docker registry, GitHub, and IBM Cloud credentials, onboard a github repository for a serving executable, create an Application and Configuration for the deployment, and deploy the serving executable on SAP AI Core. Once the serving runtime is succesfully deployed, we test its endpoints using sample data.

You will learn:

- How to connect to SAP AI Core with Python SDK
- How to create a workflow for a serving runtime
- How to deploy a custom ML engine and make REST calls to it's endpoints

### Prerequisites

- [SAP Businness Technology Platform (BTP)](https://help.sap.com/docs/btp) services and applications
    - [SAP AI Core](https://discovery-center.cloud.sap/serviceCatalog/sap-ai-core?region=all)
    - [SAP AI Launchpad](https://discovery-center.cloud.sap/serviceCatalog/sap-ai-launchpad?region=all)
- Access to a Jupyter Notebook environment, such as a [project](https://www.ibm.com/docs/en/watsonx?topic=projects) in watsonx
- A personal access token for your Docker registry
- A [GitHub repository](https://docs.github.com/en/get-started/quickstart/create-a-repo) and [personal access token (PAT)](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) to access the repository

## Table of Contents

1.  [Step 1: Set up SAP AI Core SDK to connect with SAP AI Core](#aicore_sdk)

1.  [Step 2: Onboard GitHub to SAP AI Core](#onboard)

1.  [Step 3: Create a generic secret for connecting to OpenScale](#generic_secret)

1.  [Step 4: Create a Docker registry secret for Docker Registry](#docker_registry)

1.  [Step 5: Create an Application](#create_application)

1.  [Step 6: Create a deployment configuration](#create_configuration)

1.  [Step 7: Start the deployment](#start_deployment)

1.  [Step 8: Make a prediction](#make_prediction)

1.  [Step 9: Make a prediction and send logging to OpenScale](#predict_and_log)

1.  [Step 10: Stop the deployment](#stop_deployment)

1.  [Summary](#summary)

Before you begin, you'll need to use your own credentials to replace what's in the examples below. To avoid sharing them accidentally, copy the following code block into a new code cell, and make it a hidden cell by adding `# @hidden_cell` to the top.

```
# Set your SAP AI Core credential
client_id = '<your_client_id>'
client_secret = '<your_client_secret>'
base_url = '<ai_api_url>' + '/v2'
auth_url = '<url>' + '/oauth/token'

# IBM Cloud API Key and OpenScale instance details to be stored in an AI Core secret
cloudapikey = '<your_ibm_cloud_api_key>'
service_instance_id = '<openscale_service_instance_id>'
subscription_id = '<openscale_subscription_id>'
service_url = '<openscale_service_url>'
secret_name = "<name_of_generic_secret>"

# Set the GitHub credential
github_cred_name = '<any_unique_name_for_the_git_repository_credential>'
github_repo_url = 'https://github.com/<your_username>/<repo_name>'
github_username = '<your_username>'
github_password = '<your_personal_assess_token>'

# Set the Docker Registry credential with your IBM Entitlement Key
entitled_registry_credential = "{\"auths\":{\"cp.icr.io\":{\"username\":\"cp\",\"password\":\"<your_key>\"}}}"
```

In [1]:
# The code was removed by Watson Studio for sharing.

<a id="aicore_sdk"></a>
### Step 1: Set up SAP AI Core SDK to connect with SAP AI Core

The SAP AI Core SDK is a Python-based SDK that lets you access SAP AI Core using Python methods and data structures. It provides tools that help you to manage your scenarios and workflows in SAP AI Core. The SAP AI Core SDK can be used to interact with SAP AI Core. It provides access to all public lifecycle and administration APIs.

The command line tool `pip` is the Python package installer. You can use `pip` to install the SAP AI Core SDK for Python:

In [2]:
# Install SAP AI Core SDK
%pip install ai-core-sdk

Note: you may need to restart the kernel to use updated packages.


Connect with SAP AI Core with your AI Core credential:

In [3]:
# Load AI Core Client Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create a client connection
ai_core_client = AICoreV2Client(
    base_url = base_url,
    auth_url = auth_url,
    client_id = client_id,
    client_secret = client_secret
)

<a id="onboard"></a>
### Step 2: Onboard GitHub to SAP AI Core

You can use your own git repository to version control your SAP AI Core templates. The GitOps onboarding to SAP AI Core instances involves setting up your git repository and synchronizing your content.

On-board a new GitHub repository:

In [4]:
ai_core_client.repositories.create(
    name = github_cred_name,
    url = github_repo_url,
    username = github_username,
    password = github_password
)

Execute the command below to check the on-boarding status:

In [5]:
response = ai_core_client.repositories.query()

for repository in response.resources:
    if repository.name == 'aicore-pipeline-openscale-demo':
        print('Name:', repository.name)
        print('URL:', repository.url)
        print('Status:', repository.status)

Name: aicore-pipeline-openscale-demo
URL: https://github.com/kevinxhuang/aicore-pipeline-openscale-demo
Status: RepositoryStatus.COMPLETED


<a id="generic_secret"></a>
### Step 3: Create a generic secret for connecting to OpenScale

In the below cell, we store the IBM Cloud API Key and OpenScale service instance details in SAP AI Core as a generic secret. The secret will be passed to the deployed serviing runtime container as environmental variables.

The secret in SAP AI Core needs to be base64 encoded. The following code cell encodes data in base64 format.

In [6]:
import json
import base64

# Define the JSON object
json_data = {
              "cloudapikey": cloudapikey,
              "service_instance_id": service_instance_id,
              "subscription_id": subscription_id,
              "service_url": service_url
            }

# Convert JSON to string
json_string = json.dumps(json_data)

# Encode the string in Base64
base64_encoded = base64.b64encode(json_string.encode('utf-8')).decode('utf-8')

The encoded data is then stored as a secret on SAP AI Core by sending a post request.

In [7]:
import requests

url = f"{base_url}/admin/secrets"
headers = {
    "Authorization": ai_core_client.rest_client.get_token(),
    "Content-Type": "application/json",
    "AI-Resource-Group": "default"
}

data = {
    "name": secret_name,
    "data": { "credentials": base64_encoded }
}

response = requests.post(url, headers=headers, json=data)

# Check the response
if response.status_code == 200:
    print("Secret created successfully")
else:
    print(f"Error creating secret. Status code: {response.status_code}")
    print(response.text)

Secret created successfully


<a id="docker_registry"></a>
### Step 4: Create a Docker registry secret

The runtime container image required to serve the custom-trained model is stored in a Docker registry. The credentials for Docker registries are managed using secrets.

Create a Docker registry secret for connecting SAP AI Core to your Docker registry:

In [8]:
response = ai_core_client.docker_registry_secrets.create(
    name = "dockerhub-secret",
    data = {
        ".dockerconfigjson": entitled_registry_credential
    }
)

print(response.__dict__)

{'message': 'secret has been created'}


<a id="create_application"></a>
### Step 5: Create an Application

SAP AI Core supports multiple hyperscaler object stores, such as Amazon S3, OSS (Alicloud Object Storage Service), SAP HANA Cloud, Data Lake and Azure Blob Storage. The connected storage stores your dataset, models and other cache files of the Metaflow Library for SAP AI Core. Storage credentials are managed using secrets. You can create multiple object store secrets.

You will create a folder in your GitHub repository connected SAP AI Core, where you will store the workflow (executable). You will then register this folder as an **Application** in SAP AI Core to enable syncing of the workflow as an executable.

You can create multiple **Applications** in SAP AI Core for syncing multiple folders. This helps you organize separate folders for storing workflows YAML files for separate use cases.

1. Create a executable YAML file named `openscale-demo/serving_executable.yaml` in your GitHub repository connected to SAP AI Core.
2. Edit and execute the code below to create an **Application** and sync the folder `openscale-demo`
3. Verify your workflow sync status, using the following code

You may use the existing GitHub path which is already tracked synced to your application of SAP AI Core.

**IMPORTANT**: The structure(schemas) of workflows and executables are different for both training and serving in SAP AI Core. For available options for the schemas you must refer to the [official help guide of SAP AI Core](https://help.sap.com/docs/AI_CORE/2d6c5984063c40a59eda62f4a9135bee/8a1f91a18cf0473e8689789f1636675a.html?locale=en-US).

```yaml
apiVersion: ai.sap.com/v1alpha1
kind: ServingTemplate
metadata:
  name: openscale-demo-240213
  annotations:
    scenarios.ai.sap.com/description: "Custom ML Engine"
    scenarios.ai.sap.com/name: "openscale-demo"
    executables.ai.sap.com/description: "Custom ML Engine executable"
    executables.ai.sap.com/name: "openscale-demo-exectuable"
  labels:
    scenarios.ai.sap.com/id: "openscale-demo"
    ai.sap.com/version: "1.0.0"
spec:
  inputs:
    parameters:
      - name: greetmessage # placeholder name
        type: string
  template:
    apiVersion: "serving.kserve.io/v1beta1"
    metadata:
      labels: |
        ai.sap.com/resourcePlan: starter # computing power
    spec: |
      predictor:
        imagePullSecrets:
          - name: dockerhub-secret
        containers:
        - name: kserve-container
          image: "kevinxhuang/demo:openscale"
          ports:
            - containerPort: 7000 # customizable port
              protocol: TCP
          command: ["/bin/sh", "-c"]
          args:
            - >
              set -e && echo "Starting" && gunicorn --chdir /app/src auto:app -b 0.0.0.0:7000
          env:
            - name: OPENSCALE_CREDS
              valueFrom:
                secretKeyRef:
                  name: openscale-secret # a generic secret name of your choice
                  key: credentials
            - name: greetingmessage # different name to avoid confusion
              value: "{{inputs.parameters.greetmessage}}"

```

**Understanding your serving executable**

- You use the `starter` computing resource plan with `ai.sap.com/resourcePlan`. To start, using a non-GPU based resource plan for serving (like `starter`) is cost effective. Find out more about available resource plans in [the help portal](https://help.sap.com/docs/AI_CORE/2d6c5984063c40a59eda62f4a9135bee/57f4f19d9b3b46208ee1d72017d0eab6.html?locale=en-US).
- You set the auto scaling of the server with the parameters: `minReplicas` and `maxReplicas`.
- You set the credentials to access the docker registry via `imagePullSecrets`. You must ensure that if you are using a public docker registry that has the file type `docker.io`, your secret points to the URL `https://index.docker.io`. You may delete and recreate the docker registry secret. This will not affect training templates running in parallel.
- You use the placeholder `env` to pass your `inputs` values as environment variables in your Docker image.
- You must set the name of the container to `kfserving-container` or `kserve-container`.

Create an `Application` in SAP AI Core.

In [9]:
response = ai_core_client.applications.create(
    application_name = "openscale-demo-app",
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "openscale-demo"
)

print(response.__dict__)

{'id': 'openscale-demo-app', 'message': 'Application has been successfully created.'}


Verify your workflow sync status:

In [10]:
import time

time.sleep(60)
response = ai_core_client.applications.get_status(application_name = 'openscale-demo-app')

print(response.__dict__)
print('*'*80)
print(response.sync_ressources_status[0].__dict__)

{'health_status': 'Healthy', 'sync_status': 'Synced', 'message': 'successfully synced (all tasks run)', 'source': <ai_core_sdk.models.application_source.ApplicationSource object at 0x7f8dbe7d1270>, 'sync_finished_at': '2024-02-14T03:38:27Z', 'sync_started_at': '2024-02-14T03:38:26Z', 'reconciled_at': '2024-02-14T03:38:28Z', 'sync_ressources_status': [<ai_core_sdk.models.application_resource_sync_status.ApplicationResourceSyncStatus object at 0x7f8dbe7d08b0>]}
********************************************************************************
{'name': 'openscale-demo-240213', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/openscale-demo-240213 created'}


After your workflows are synced, your `Scenario` will be automatically created in SAP AI Core. The name and ID of the scenario will be same as the one mentioned in your workflows. After The syncing, your workflow will be recognized as an executable.

<a id="create_configuration"></a>
### Step 6: Create a deployment configuration

Here are the important pieces of your configuration:

- The `scenario_id` should contain the same value as in your executable.
- The `executable_id` is the name key of your executable.

In [11]:
# Replace the artifact_id field value with your own ID, then execute the code.
from ai_core_sdk.models import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "openscale-demo-conf",
    resource_group = "default",
    scenario_id = "openscale-demo",
    executable_id = "openscale-demo-240213"
)

print(response.__dict__)
conf_id=response.__dict__['id']

{'id': '7b57612b-cb89-46b3-94eb-8ca3e31f2f30', 'message': 'Configuration created'}


<a id="start_deployment"></a>
### Step 7: Start the deployment

Execute the code to start the deployment:

In [12]:
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = conf_id
)
print(response.__dict__)
deploy_id=response.__dict__['id']

{'id': 'd2131b420b9a3efc', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


<div class="alert alert-block alert-warning">
<b>Important:</b>

Note the unique ID generated of your deployment. You may create multiple deployments using the same configuration ID, each of which will have a unique endpoint.

</div>

Check deployment status:

In [13]:
while str(response.status)!='Status.RUNNING':

    response = ai_core_client.deployment.get(
        resource_group = 'default',
        deployment_id = deploy_id
    )

    print("Status: ", response.status)
    print('*'*80)
    time.sleep(30)

print(response.__dict__)

Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
*******************************

<div class="alert alert-block alert-info">

<b>Tip:</b>

This may take a few minutes for the status to change: `UNKNOWN` > `PENDING` > `RUNNING`.

</div>

Now query the deployment logs to view its output:

In [14]:
from ai_core_sdk.models import TargetStatus

response = ai_core_client.deployment.query_logs(
    resource_group = "default",
    deployment_id = deploy_id
)

for log in response.data.result:
    print(log.msg)
    print("---")

Starting
---
[2024-02-14 03:43:58 +0000] [7] [INFO] Starting gunicorn 21.2.0
---
[2024-02-14 03:43:58 +0000] [7] [INFO] Listening at: http://0.0.0.0:7000 (7)
---
[2024-02-14 03:43:58 +0000] [7] [INFO] Using worker: sync
---
[2024-02-14 03:43:58 +0000] [8] [INFO] Booting worker with pid: 8
---


<a id="make_prediction"></a>
### Step 8: Make a prediction

In [15]:
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/" + deploy_id
deployment_url

'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d2131b420b9a3efc'

In [16]:
import requests

# URL
endpoint = f"{deployment_url}/v2/greet" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "AI-Resource-Group": "default"}
response = requests.get(endpoint, headers=headers)

print(response.text)

Flask Code: Model is loaded.


In [17]:
# Prepare a sample input
test_input = {'input_data': [{'fields': ['no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'kpis_met_above_80_percent', 'any_awards_won', 'avg_training_score', 'department_Finance', 'department_HR', 'department_Legal', 'department_Operations', 'department_Procurement', 'department_R&D', 'department_Sales & Marketing', 'department_Technology', 'education_Below Secondary', "education_Master's & above", 'gender_m'],'values': [[1,29,1.0,1,0,0,49,0,0,0,0,0,0,1,0,0,0,0]]}]}
#records = {'input_data': [{'fields':df_logging.columns.tolist(),'values':df_logging.values.tolist()}]}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "AI-Resource-Group": "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)
#response = requests.post(endpoint, headers=headers, json=records)

print(response)
print( response.json())

<Response [200]>
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.99, 0.01]]]}]}


<a id="predict_and_log"></a>
### Step 9: Make a prediction and send logging to OpenScale

In [18]:
endpoint = f"{deployment_url}/v2/predict_and_log" # endpoint implemented in serving engine

headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "AI-Resource-Group": "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print(response)
print( response.json())

<Response [200]>
{'logging_response': 'Payload Logging and Feedback logging Successful', 'model_prediction': {'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.99, 0.01]]]}]}}


<a id="stop_deployment"></a>
### Step 10: Stop the deployment

A running deployment incurs cost because it is allocated cloud resources. Stopping the deployment frees up these resources and therefore there is no charge for a deployment of status `Stopped`.

In [19]:
from ai_core_sdk.models import TargetStatus

response = ai_core_client.deployment.modify(
    resource_group = "default",
    deployment_id = deploy_id,
    target_status = TargetStatus.STOPPED
)

print(response.__dict__)

{'id': 'd2131b420b9a3efc', 'message': 'Deployment modification scheduled'}


<div class="alert alert-block alert-info">

<b>Tip:</b>

You cannot restart a deployment. You must create a new deployment, reusing the configuration. Each deployment will have a different URL.
</div>

<a id="summary"></a>
## Summary

Now we've demonstrated how you can deploy an IBM Watson OpenScale Custom ML Engine on SAP AI Core and securely make REST calls to its endpoints to make predictions and then store request and response data to payload and feedback logging tables in IBM Watson OpenScale.